# Handwritten dataset

In [90]:
%run "../config/local.ipynb"
%run "../utils/functions.ipynb"

%config IPCompleter.greedy=True

import numpy as np
import pandas as pd
import sklearn
from sklearn.model_selection import StratifiedKFold
from src.mrcnn.utils import Dataset

## Dataset class

In [125]:
class HandwrittenDataset(Dataset):    
    
    def __init__(self, 
                images_path, 
                labels_file_path,
                classes=None,
                stage: str = "train",
                img_ids: np.array = None,
                transforms=None,
                size=(512,512)):
        super().__init__()                
        # Add classes
        self.init_classes(classes)
        # input files paths        
        self.images_dir_path = images_path
        self.labels_file_path = labels_file_path
        # images ids         
        self.img_ids = img_ids
        # data transformation function
        self.transforms = transforms
        # load referenced masks        
        self.load_masks_references()        
        # load the images 
        self.load_images()
        
    def init_classes(self, classes):
        """ add classes """
        for item in classes:
            self.add_class(item['source'], int(item['num']), item['name'])                            
            
    def load_images(self):
        """ Generate the images list
        """
        # Add images
        for idx in range(len(self.img_ids)):        
            self.add_image("pages", image_id=idx, path=os.path.join(self.images_dir_path, self.img_ids[idx]), name=self.img_ids[idx])        
    
    def load_masks_references(self): 
        # load all masks
        all_masks = pd.read_csv(self.labels_file_path)        
        all_masks = all_masks.set_index(['name', 'num'])                
        
        # filter the masks by images ids
        self.masks_references = all_masks.loc[self.img_ids].index.get_level_values(['name', 'num'])
        
        print("finish load_masks_references")
        
    def load_mask(self, image_id):
        """Generate instance masks for shapes of the given image ID.
        """
        img = self.image_info[image_id]
        
        # image name
        name = img['name']
        
        # check if the image is referenced
        if not name in self.masks_references.index:
            return None, []
        
        # list of labels     
        labels = self.masks_references.loc[name]['label']
        
        if type(labels) is str:
            labels = [labels]
        else:
            labels = list(labels)
        
        masks = np.array(list(refs.loc[name]['mask'].apply(lambda v: rle_to_mask(v, size[0], size[1]))))
                
        # map the labels to the class indexes
        class_ids = np.array([self.class_names.index(label) for label in labels])    
        
        # create the masks tensor
        masks_tensor = np.array([mask.astype(np.bool) for mask in masks])
                
        return np.stack(masks_tensor,axis=-1), class_ids.astype(np.int32)
    
    def get_random_image(self, min_label_count=0):
        """ return a filename and a (widht, height, RGB) array of a randmly selected image
        with a label !
        """
        # idx = random.randint(0,len(self.image_info)-1)   
        # img = self.image_info[idx]
        
        # pick an image id from the masks references 
        # refs_idx = random.randint(0,len(self.masks_references.index)-1)   
        # img_id = self.masks_references.index[refs_idx]
        
        # list of multi-label image
        df = self.masks_references.reset_index()
        df_a = df.groupby('name').count() >= min_label_count
        multi_labels = list(df_a[df_a['label']].index)

        # random selection of an image
        ref_idx = random.randint(0,len(multi_labels))    
        img_id = multi_labels[ref_idx]
        
        idx = [info['id'] for info in self.image_info if info['name'] == img_id][0]        
                
        return idx, self.load_image(idx)
    
    def get_transformation(self, image_id):
        # load the image
        img = self.load_image(image_id)
        # img = img.astype(np.uint8)
        
        # get the mask
        masks, class_ids = self.load_mask(image_id)
        
        # image name        
        img_infos = self.image_info[image_id]
        img_name = img_infos['name']
        
        # img = img.astype(np.uint8)
        
        data = {"image": img} 
        
        # add each mask for the transformation
        for i in range(len(class_ids)):
            idx = 'mask{}'.format(i)
            mask = masks[:,:,i].astype(np.uint8)
            data[idx] = mask    
        
        # m = masks[:,:,0].astype(np.uint8)
        
        # apply augementation
        transformer = self.__get_training_transformer(data)        
        augmented = transformer(**data)

        # augmented image
        img = augmented['image']
        
        # list of augmented masks
        tranformed_masks = [augmented[key].astype(bool) for key in augmented.keys() if not key == 'image']
        
        return img, np.stack(tranformed_masks,axis=-1)
    
    def __get_training_transformer(self, data):
        """ Return  the albumentation transformation function with respect to the number of masks         
        """
        train_transform = [
            albu.HorizontalFlip(p=0.5),
            albu.VerticalFlip(p=0.5),
            albu.Blur(p=0.5),
            albu.ShiftScaleRotate(
                scale_limit=0.5,
                rotate_limit=0,
                shift_limit=0.1,
                p=0.5,
                border_mode=0
            ),
            albu.GridDistortion(p=0.5),        
        ]
        
        target = {}
        for i in range(len(data.keys()) -1):
            target['mask' + str(i)] = 'mask'            
        return albu.Compose(train_transform, additional_targets=target)   

## Train test split

In [126]:
def train_valid_split(masks_path, fold_num=0, n_folds=4, seed=42):
    """
    return train and validation ids from the label file
    parameters:
        model_no: the model number if you train multiples models with the same data
        n_folds: number of folds to create 
    """
    
    df_masks = pd.read_csv(masks_path)
                
    # get count of label by file
    refs = df_masks.groupby('name').count().reset_index()
    
    # define the kfold generator
    skfolds = StratifiedKFold(n_splits=n_folds,random_state=seed)
    
    # create the folds
    folds = [[train_idxs, valid_idxs] for train_idxs, valid_idxs in skfolds.split(refs['name'], refs['label'])]
    sampled_train_ids = list(refs.iloc[folds[fold_num][0]]['name'])
    sampled_valid_ids = list(refs.iloc[folds[fold_num][1]]['name'])
    
    return sampled_train_ids, sampled_valid_ids

## Check dataset 

In [127]:
train_ids, valid_ids = train_valid_split(MASKS_FILE)
print("train_ids: {} items, valid_ids: {} items".format(len(train_ids), len(valid_ids)))

train_ids: 131 items, valid_ids: 44 items


/home/laurent/workspaces/mooke/github/dl-handwritting-recognition/p_env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/home/laurent/workspaces/mooke/github/dl-handwritting-recognition/p_env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  % (min_groups, self.n_splits)), UserWarning)


## Build train and valid datasets

In [128]:
def build_train_valid_dataset(images_path, masks_path, classes, seed=42):
    # get train and validation ids    
    train_ids, valid_ids = train_valid_split(masks_path=masks_path, seed=seed)
    
    # Training dataset
    dataset_train = HandwrittenDataset(images_path=images_path,
                                  labels_file_path=masks_path,
                                  classes=classes,
                                 img_ids=train_ids,
                                 transforms=get_training_augmentation())
    dataset_train.prepare()
    
    # Validation dataset
    dataset_val = HandwrittenDataset(images_path=images_path, 
                                labels_file_path=masks_path,
                                classes=classes,
                                img_ids=valid_ids,
                                transforms=get_training_augmentation())
    dataset_val.prepare()
    
    return dataset_train, dataset_val

## Check datasets

In [129]:
classes = [{'source':'clouds', 'num':1, 'name':'Fish'}]
train_ds, val_ds = build_train_valid_dataset(RESIZED_512x512_FEATURES_DIR, MASKS_FILE, classes)

/home/laurent/workspaces/mooke/github/dl-handwritting-recognition/p_env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning
/home/laurent/workspaces/mooke/github/dl-handwritting-recognition/p_env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=4.
  % (min_groups, self.n_splits)), UserWarning)


KeyError: "Level ['name', 'num'] not found"

In [ ]:
train_ds.get_random_image()